In [ ]:
from crawlbase import CrawlingAPI
from bs4 import BeautifulSoup
import pickle
import pandas as pd
import time
from openai import OpenAI
import os
import reddit_scrape

In [7]:
def process(doc):
    client = OpenAI(api_key = os.environ["OPENAI_API_KEY"])
    response = client.responses.create(
        model='gpt-5.1',
        input="is this post suicidal?" + " '" + doc + "' " + 'return True or False'
    )
    if 'True' in response.output_text:
        return True
    else: return False

In [ ]:
def save_post(html_data,sub_red):    
    soup = BeautifulSoup(html_data, 'html.parser')
    instances = soup.find_all('div', property="schema:articleBody")
    docs = []
    for instance in instances:
        p_tags = []
        for p_tag in instance:
            if p_tag:
                stripped = p_tag.get_text().strip()
                p_tags.append(stripped)
        docs.append(''.join(p_tags))


    verdicts = pd.read_csv('verdicts.csv')
    for doc in docs:
        bool = process(doc) ### Chatgpt
        if not (verdicts['post'] == doc).any() and doc:
            verdicts = pd.concat([verdicts, pd.DataFrame({'post':doc,'verdict':bool, 'sub_red':sub_red},index=[0])])
            verdicts.to_csv('verdicts.csv', index=False)
            print('saved a post from r/' + sub_red + ' ' + str(bool))
# save_post(html_data,'depression')



saved a post from r/depression True
saved a post from r/depression True
saved a post from r/depression True


In [ ]:
def main():
    sub_reds = ['depression', 'sad', 'confessions']
    # sub_reds = ['depression']
    iterations = 0
    # while iterations < 24:

    for sub_red in sub_reds:
        try:

            save_post(reddit_scrape(sub_red),sub_red)

        except Exception as e: print(sub_red, e)

        # time.sleep(10)
    # time.sleep(1800)

    iterations += 1

In [ ]:
# verdicts = pd.DataFrame({'sub_red': [],'verdict':[], 'post':[]})
# verdicts.to_csv('verdicts.csv')